# Conexão com o JUSBR 

Este notebook demonstra como autenticar e obter um token de acesso para o portal JUSBR usando a classe `JUSBR_Scraper`.


In [1]:
import juscraper as jus
import json

jusbr = jus.scraper("jusbr")

## Autenticação manual via gov.br

1. Acesse https://www.jus.br
2. Faça login usando o gov.br
3. Entre na página https://portaldeservicos.pdpj.jus.br/consulta (ou clique no botão 'Consultar processo', que aparece após o login)
4. Na nova página, abra a aba Network do navegador (F12 ou 'Inspecionar elemento')
5. Atualize a página (F5 ou ctrl+R ou no botão atualizar)
6. Nas requisições que vão aparecer, procure a requisição que tem nome 'token'. Clique nela.
7. Na tela ao lado, clique em 'Resposta'.
8. Selecione e copie o campo 'access_token' que aparece lá.


In [3]:
import os
from dotenv import load_dotenv
from getpass import getpass

load_dotenv()

access_token = os.environ.get("JUSBR_ACCESS_TOKEN")
if access_token is None:
    # input como senha oculta
    access_token = getpass("Digite o token JWT: ")


In [ ]:

jusbr.auth(token=access_token)

True

Alternativa de autenticação: Firefox

Faça o login no firefox e use o código abaixo, que captura os cookies e usa para obter um token JWT.

In [2]:
jusbr.auth_firefox()

True

## Baixando metadados dos processos

Baixando uma lista de casos:

In [3]:
processos = ['10092161720238260016', '00039417120248260048']

resultados = jusbr.cpopg(processos)

resultados

,processo,numeroProcesso,idCodexTribunal,detalhes
0,10092161720238260016,1009216-17.2023.8.26.0016,None,"{'nivelSigilo': 0, 'siglaTribunal': 'TJSP', 'n..."
1,00039417120248260048,0003941-71.2024.8.26.0048,None,"{'nivelSigilo': 0, 'siglaTribunal': 'TJSP', 'n..."


In [4]:
result_list = resultados['detalhes'].iloc[0]

print(json.dumps(result_list['tramitacaoAtual']['grau'], indent=2, ensure_ascii=False))

{
  "sigla": "G1",
  "nome": "1° Grau",
  "numero": 1
}


Se quiser visualizar o texto do caso:

## Baixando autos dos processos

In [5]:
df_docs = jusbr.download_documents(resultados)

print(df_docs.shape)
df_docs.head(3)

(6, 17)


,numero_processo,idCodex,sequencia,nome,tipo,dataHoraJuntada,nivelSigilo,hrefTexto,hrefBinario,texto,_raw_text_api,arquivo,idOrigem,idDocumento,descricao,tipoDocumento,dataJuntada
0,1009216-17.2023.8.26.0016,23219022228,28,Sentenas.pdf,"{'codigo': 550, 'nome': 'Sentença', 'idCodex':...",2024-12-02T19:51:05,PUBLICO,/processos/1009216-17.2023.8.26.0016/documento...,/processos/1009216-17.2023.8.26.0016/documento...,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\nCO...,\nTRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\n...,"{'tipo': 'text/richtext', 'quantidadePaginas':...",PG5JM_336374331_0,None,None,None,None
1,1009216-17.2023.8.26.0016,22210997949,24,Ato_Ordinatrio.pdf,"{'codigo': 14575, 'nome': 'Ato Ordinatório', '...",2024-04-24T11:54:33,PUBLICO,/processos/1009216-17.2023.8.26.0016/documento...,/processos/1009216-17.2023.8.26.0016/documento...,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\nCO...,\nTRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\n...,"{'tipo': 'text/richtext', 'quantidadePaginas':...",PG5JM_306567040_0,None,None,None,None
2,1009216-17.2023.8.26.0016,16542603918,10,Ato_Ordinatrio.pdf,"{'codigo': 14575, 'nome': 'Ato Ordinatório', '...",2023-05-25T14:31:12,PUBLICO,/processos/1009216-17.2023.8.26.0016/documento...,/processos/1009216-17.2023.8.26.0016/documento...,TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\nCO...,\nTRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO\n...,"{'tipo': 'text/richtext', 'quantidadePaginas':...",PG5JM_262181327_0,None,None,None,None


In [6]:
print(df_docs['texto'][0][:200])

TRIBUNAL DE JUSTIÇA DO ESTADO DE SÃO PAULO
COMARCA de SÃO PAULO
Foro Central Juizados Especiais Cíveis
2ª Vara do Juizado Especial Cível - Vergueiro
Rua Vergueiro, 835, São Paulo - SP - cep 01504-001 


## Baixando uma lista grande de processos

Se a lista de processos é muito grande, pode ser uma boa prática baixar um processo por vez e salvar os resultados em um arquivo parquet, por exemplo.

No script abaixo, criamos uma pasta dentro da pasta atual, chamada jusbr/cpopg/. Ali dentro, para cada processo, criamos uma pasta com o número do processo e salvamos 2 arquivos: 1 com os metadados daquele processo e 1 com os textos dos documentos. A chave para juntar as duas bases é o número do processo.

In [ ]:
import os
import pandas as pd
from tqdm import tqdm
from joblib import Parallel, delayed

processos = ['10092161720238260016', '00039417120248260048', '10226951920238260003']

resultados = jusbr.cpopg(processos)

In [ ]:

def download_document(proc_no, jusbr, verbose=True):
    if verbose:
        print(f"Baixando documentos do processo {proc_no}\n")
    
    resultado = jusbr.cpopg([proc_no])
    row = resultado.iloc[0]
    processo = row['processo']
    row_df = pd.DataFrame([row])
    # cria a pasta
    process_path = os.path.join('jusbr/cpopg', processo)
    os.makedirs(process_path, exist_ok=True)
    
    # baixa documentos do processo
    if verbose:
        print(f"Baixando autos do processo {proc_no}\n")
    
    df_docs = jusbr.download_documents(row_df)
    #df_docs = pd.DataFrame()
    # salva documentos
    df_docs.to_parquet(os.path.join(process_path, 'documents.parquet'), index=False)
    # salva metadata
    row_df.to_parquet(os.path.join(process_path, 'metadata.parquet'), index=False)


Parallel(n_jobs=20)(
    delayed(download_document)(proc_no, jusbr)
    for proc_no in processos
)
